# import data

In [ ]:
import pandas as pd
import numpy as np
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

In [ ]:
df_origin = pd.read_excel('stocks.xlsx')
df_contract_value =  df_origin.set_index('date')[['lastUpdateTime','Unnamed: 82']].reset_index()
df_contract_value.columns = ['code','value','industry']
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_wind =  w.wss(f"{stk_code_str}", "sec_name", "tradeDate=20221102;",usedf=True)[1]
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_info = df_contract_value.set_index('code').loc[stk_code_list]
# pd.set_option('display.max_rows', None)
df_stk_info['name'] = df_stk_wind['SEC_NAME']
df_stk_clean = df_stk_info[df_stk_info['industry']!=0].fillna('工业').sort_values(by='industry').reset_index()
df_stk_clean = df_stk_clean[['code','name','value','industry']]
# pd.set_option('display.max_rows', None)
df_stk_clean['value'] = df_stk_clean['value'].apply(lambda x: format(x, 'f'))

tmp_str = ','.join(df_stk_clean['code'].tolist())
df_ipo_date = w.wss(tmp_str, "ipo_date", "",usedf=True)[1].reset_index()
df_stk_clean['IPO_DATE'] = df_ipo_date['IPO_DATE']
# df_stk_clean.to_excel('stock_hold.xlsx')

In [ ]:
# pd.set_option('display.max_rows', None)
df_stk_clean

In [ ]:
idsty_stk_dic = {}
for index,row in df_stk_clean.iterrows():
    tmp_code = row['code']
    tmp_industry = row['industry']
    if tmp_industry not in idsty_stk_dic:
        idsty_stk_dic[tmp_industry] = [tmp_code]
    else:
        idsty_stk_dic[tmp_industry].append(tmp_code)
print(idsty_stk_dic.keys())
df_industry_count = pd.DataFrame(df_stk_clean['industry'].value_counts())
df_industry_count = df_industry_count.reset_index()
# df_industry_count.to_excel('industry.xlsx')
df_industry_count

# industry setting

In [ ]:
target_industry = '交通运输'
df_stk_clean[df_stk_clean['industry']==target_industry]

# return calculate

In [ ]:
# cbond_list = ['113516.SH','110043.SH',
# ]
cbond_list = idsty_stk_dic[target_industry]  
# ###############################################################
before_trade_date = '20201029'#监测当日前一个交易日
present_date = '20201030' #监测当日
# before_trade_date = '20201026'#监测当日前一个交易日
# present_date = '20201027' #监测当日
# next_trade_date = '20220917' #监测当日下一个自然日
# ###############################################################

start_date = pd.to_datetime(before_trade_date) 

end_date = pd.to_datetime('20221030') #多一个月没关系，切片的时候，不需要这多出来的一个月的对子会切掉

cbond_price = pd.DataFrame([])
for code in cbond_list:
    price_df = w.wsi(code, "close", start_date, end_date,usedf=True)[1]   #只要了收盘价
    price_df.columns = [code]
    cbond_price = pd.concat([cbond_price,price_df],axis=1)
cbond_price = cbond_price.ffill()
# cbond_price = cbond_price[cbond_price.index.time!=time(15,0,0)]
# cbond_price = cbond_price.loc['2022-09-09 15:00:00':] #从前一个交易日的收盘价开始截取
cbond_price_begin_time = before_trade_date[:4]+'-' + \
                    before_trade_date[4:6]+'-' + \
                    before_trade_date[6:8]+' 15:00:00'
cbond_price = cbond_price.loc[cbond_price_begin_time:]
cbond_price

In [ ]:
# cbond_return_2 = pd.DataFrame(columns=cbond_list)
# # tmp_stk_code = '601006.SH'
# # cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
# stock_preclose_2 = cbond_price.iloc[0,:] # 初始时刻的stock前收盘价
# stock_preclose_2
# for index,row in cbond_price.iterrows():
#     # cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
#     cbond_return_2.loc[index] = (row/stock_preclose_2-1).tolist()
#     if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
#         # cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
#         stock_preclose_2 = row
# cbond_return_2        

In [ ]:
cbond_return = pd.DataFrame([])
# tmp_stk_code = '601006.SH'
# cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_price.iloc[0,:].tolist() # 初始时刻的stock前收盘价

for index,row in cbond_price.iterrows():
    
    # cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    for i in range(len(cbond_list)):
        cbond_return.loc[index,cbond_list[i]] = row[cbond_list[i]]/stock_preclose[i] - 1
        if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
            # cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
            stock_preclose[i] = row[cbond_list[i]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

In [ ]:
# cbond_return = cbond_return.dropna()
cbond_return

In [ ]:
cbond_return.isnull().sum(axis=0)  # 按列统计

In [ ]:
# cbond_return = cbond_return.dropna()

# correlation & cointegration test

In [ ]:
# print(cbond_return.corr(method='pearson').describe())
pd.set_option('display.max_columns',None)
df_corr = cbond_return.corr(method='pearson')
for i in range(len(df_corr)):
    df_corr.iloc[i,i]=np.nan
tmp_se = df_corr.max()
# df_corr
print('max corr code:',f'\'{tmp_se.idxmax()}\'')
tmp_se.sort_values(ascending=False)

In [ ]:
main_code = '000422.SZ'
tmp_line = df_corr.loc[main_code]
tmp_line = pd.DataFrame(tmp_line)
for index,row in tmp_line.iterrows():
    if row[main_code] == tmp_se.loc[main_code]:
        # print(index)
        sec_code = index
        tmp_corr = tmp_se.loc[main_code]
print(f'\'{main_code}\',\'{sec_code}\'','@',tmp_corr)

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)
pair_code = [main_code,sec_code]
x_code = pair_code[0]
y_code = pair_code[1]   
x = cbond_price[x_code].tolist()
y = cbond_price[y_code].tolist()

cointegration_test(y, x),cointegration_test(x, y)

# output to pickle

In [ ]:
save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/price/{target_industry}.pkl'
cbond_price.to_pickle(save_path)
save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/return/{target_industry}.pkl'
cbond_return.to_pickle(save_path)
target_industry